<a href="https://colab.research.google.com/github/Maxxx-VS/Machine_Learning/blob/main/Igor_testovoe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RestorauntGroup - адрес магазина
UniqOrderId - Количество уникальных чеков, шт
DishDiscountSumInt.average - Сумма среднего чека, р
OpenDate.Typed - Дата
DishDiscountSumInt - Сумма продаж, р
DishAmountInt - Количество проданной продукции, шт
OrderItems - технологический столбец (количество строк фиксации продаж)

Задание- спрогнозировать сумму прода и количество штук продукции на год вперед.

In [1]:
# импортирую необходимые библиотеки
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# загружаю ДФ
df = pd.read_csv('/content/danniepol2.csv', engine='python', encoding='utf-8')

In [3]:
# вывожу форму ДФ
print(df.shape)

(252080, 7)


In [4]:
# вывожу список всех столбцов
print(df.columns)

Index(['RestorauntGroup', 'UniqOrderId', 'DishDiscountSumInt.average',
       'OpenDate.Typed', 'DishDiscountSumInt', 'DishAmountInt', 'OrderItems'],
      dtype='object')


In [5]:
# удаляю технический стобец
del df['OrderItems']

In [6]:
# даю нормальные имена столбцам
df = df.rename(columns={'RestorauntGroup': 'ADRESS',
                        'UniqOrderId': 'COUNT_CHEK',
                        'DishDiscountSumInt.average': 'SUM_AVERAGE_CHEK',
                        'OpenDate.Typed': 'DATE',
                        'DishDiscountSumInt': 'SUMMA_PROD',
                        'DishAmountInt': 'COUNT_PROD'})

In [7]:
print(df)

                                                 ADRESS  COUNT_CHEK  \
0                        Ставрополь, ул. Доваторцев 75а        71.0   
1                        Ставрополь, ул. Доваторцев 75а        64.0   
2                        Ставрополь, ул. Доваторцев 75а        59.0   
3                        Ставрополь, ул. Доваторцев 75а        68.0   
4                        Ставрополь, ул. Доваторцев 75а        65.0   
...                                                 ...         ...   
252075         Исилькуль,ЛАВАШ, ул. Коммунитическая, 33        26.0   
252076        Куйбышев, ЛАВАШ,  ул.Коммунистическая, 53        17.0   
252077       Омск, ЛАВАШ, ул.21 амурская, 43,3 литера А        20.0   
252078  Омская обл, рп.Москаленки, ул.Комсомольская, 80        12.0   
252079                      Тара, ЛАВАШ, ул. Ленина, 68        13.0   

        SUM_AVERAGE_CHEK        DATE  SUMMA_PROD  COUNT_PROD  
0                 619.56  2024-02-25     43989.0       603.0  
1                 438

In [8]:
# вывожу информацию о ДФ
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252080 entries, 0 to 252079
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ADRESS            251641 non-null  object 
 1   COUNT_CHEK        252079 non-null  float64
 2   SUM_AVERAGE_CHEK  252078 non-null  float64
 3   DATE              252077 non-null  object 
 4   SUMMA_PROD        252077 non-null  float64
 5   COUNT_PROD        252077 non-null  float64
dtypes: float64(4), object(2)
memory usage: 11.5+ MB
None


In [9]:
# заполняю пропуски медианными значениями
median_value = df['COUNT_CHEK'].median()
df['COUNT_CHEK'].fillna(median_value, inplace=True)

median_value = df['SUM_AVERAGE_CHEK'].median()
df['SUM_AVERAGE_CHEK'].fillna(median_value, inplace=True)

median_value = df['SUMMA_PROD'].median()
df['SUMMA_PROD'].fillna(median_value, inplace=True)

median_value = df['COUNT_PROD'].median()
df['COUNT_PROD'].fillna(median_value, inplace=True)

<ipython-input-9-81d973cc78e3>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['COUNT_CHEK'].fillna(median_value, inplace=True)
<ipython-input-9-81d973cc78e3>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [10]:
# удалаю строки со значением NaN
df = df.dropna(subset=['DATE'])
df = df.dropna(subset=['ADRESS'])

In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 251639 entries, 0 to 252079
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ADRESS            251639 non-null  object 
 1   COUNT_CHEK        251639 non-null  float64
 2   SUM_AVERAGE_CHEK  251639 non-null  float64
 3   DATE              251639 non-null  object 
 4   SUMMA_PROD        251639 non-null  float64
 5   COUNT_PROD        251639 non-null  float64
dtypes: float64(4), object(2)
memory usage: 13.4+ MB
None


In [12]:
# считаю сколько уникальных дней (дат) собрано в ДФ
count_unicue_date = df["DATE"].unique()
print(len(count_unicue_date))
# повторно проверяю кол-во дней
df.nunique()

656


,0
ADRESS,697
COUNT_CHEK,540
SUM_AVERAGE_CHEK,73425
DATE,656
SUMMA_PROD,105421
COUNT_PROD,8055


In [19]:
# РЕШЕНИЕ №1 ЧЕРЕЗ АРИФМЕТИЧЕСКОЕ ВЫЧИСЛЕНИЕ СРЕДНЕГО ЗНАЧЕНИЯ В ДЕНЬ И ЭКСТРАПОЛЯЦИИ НА ГОД
# считаю сумму по полю, делю на количество дат и умножаю на 365
# кажется это решение - КРИВОЕ
summ_prod_df = df['SUMMA_PROD'].sum()
count_prod_df = df['COUNT_PROD'].sum()

print(f'Прогноз годовой суммы прода = {round(summ_prod_df / 656 * 360, 2)} руб.')
print(f'Прогноз годового кол-ва штук = {round(count_prod_df / 656 * 360, 2)} шт.')

Прогноз годовой суммы прода = 7578225073.52 руб.
Прогноз годового кол-ва штук = 142190477.22 шт.


In [25]:
# считаю среднее значение по полю, и умножаю на 365
# кажется это решение - ВЕРНОЕ
mean_A = df['SUMMA_PROD'].mean()
print(f'Прогноз годовой суммы прода = {round(mean_A * 365, 2)} руб.')

mean_B = df['COUNT_PROD'].mean()
print(f'Прогноз годового кол-ва штук = {round(mean_B * 365, 2)} шт.')

Прогноз годовой суммы прода = 20030129.27 руб.
Прогноз годового кол-ва штук = 375825.95 шт.


In [34]:
# РЕШЕНИЕ №2 ЧЕРЕЗ ПОЛНОСВЯЗНУЮ НС
data = df.copy()

from keras import models
from keras import layers
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow import keras

x_data = data[['COUNT_CHEK', 'SUM_AVERAGE_CHEK']]
y_data = data[['SUMMA_PROD']]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=42)
model = models.Sequential()
model.add(layers.Dense(10, activation='relu', input_shape=(5,)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='linear'))
EPOCHS = 10
BATCH_SIZE = 43
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.05)
model.save("/content/OEMZ_1.keras")
test_accuracy = model.evaluate(X_test, y_test)
print(round(test_accuracy[1], 2))


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_6" is incompatible with the layer: expected axis -1 of input shape to have value 5, but received input with shape (None, 2)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 2), dtype=float32)
  • training=True
  • mask=None